In [1]:
import os
import pandas as pd
import yfinance as yf

# ---------------------------------------------------------------------
# 1. Settings
# ---------------------------------------------------------------------

# Folder where the CSV will be saved (relative to this notebook)
DATA_DIR = "."

# Example hedge-fund-like ETF.
# You can change this ticker to any liquid alternatives / hedge fund replication ETF you prefer.
HF_TICKER = "QAI"   # IQ Hedge Multi-Strategy Tracker ETF (example)

START_DATE = "2005-01-01"

# ---------------------------------------------------------------------
# 2. Ensure data folder exists
# ---------------------------------------------------------------------

os.makedirs(DATA_DIR, exist_ok=True)

# ---------------------------------------------------------------------
# 3. Download price data from Yahoo Finance
# ---------------------------------------------------------------------

hf_raw = yf.download(
    HF_TICKER,
    start=START_DATE,
    interval="1mo",
    auto_adjust=True,   # prices already adjusted for splits/dividends
    progress=False
)

if hf_raw.empty:
    raise ValueError(
        f"No data returned for ticker {HF_TICKER}. "
        "Check the ticker symbol or your internet connection."
    )

# ---------------------------------------------------------------------
# 4. Compute simple monthly returns
# ---------------------------------------------------------------------

# With auto_adjust=True, we use 'Close'
hf_raw["HF_RET"] = hf_raw["Close"].pct_change()

# Keep only the return series and drop the first NaN
hf_returns = hf_raw[["HF_RET"]].dropna()

# Reset index and rename date column to 'Date' for consistency
hf_returns = hf_returns.reset_index().rename(columns={"Date": "Date"})

# ---------------------------------------------------------------------
# 5. Save to CSV in the data/ folder
# ---------------------------------------------------------------------

output_path = os.path.join(DATA_DIR, "hf_index.csv")
hf_returns.to_csv(output_path, index=False)

print(f"Saved hedge-fund-like ETF data to: {output_path}")
hf_returns.head()

Saved hedge-fund-like ETF data to: .\hf_index.csv


Price,Date,HF_RET
Ticker,,
0,2009-04-01,0.022882
1,2009-05-01,0.027865
2,2009-06-01,-0.003436
3,2009-07-01,0.015326
4,2009-08-01,-0.004151


In [2]:
import os
import pandas as pd
import yfinance as yf

DATA_DIR = "."           # salvar na própria pasta data/
START_DATE = "2005-01-01"

# ---------------------------------------------------------------------
# Market and risk-free proxies
# ---------------------------------------------------------------------

MKT_TICKER = "ACWI"   # MSCI ACWI ETF (global equity)
RF_TICKER  = "BIL"    # 1-3 month T-Bill ETF (risk-free proxy)

mkt_raw = yf.download(
    MKT_TICKER,
    start=START_DATE,
    interval="1mo",
    auto_adjust=True,
    progress=False
)

rf_raw = yf.download(
    RF_TICKER,
    start=START_DATE,
    interval="1mo",
    auto_adjust=True,
    progress=False
)

# Monthly returns
mkt_ret = mkt_raw["Close"].pct_change()
rf_ret  = rf_raw["Close"].pct_change()

ff = pd.concat([mkt_ret, rf_ret], axis=1)
ff.columns = ["MKT", "RF"]
ff = ff.dropna()

# Market excess return
ff["MKT_RF"] = ff["MKT"] - ff["RF"]

# Placeholders for size, value, momentum (0.0 for now).
# Later you can replace these with real Fama–French factors if you want.
ff["SMB"] = 0.0
ff["HML"] = 0.0
ff["MOM"] = 0.0

# Reorder columns to match what the main notebook expects
ff = ff[["MKT_RF", "SMB", "HML", "MOM", "RF"]]

# Save to CSV
ff_to_save = ff.reset_index().rename(columns={"Date": "Date"})
output_path_ff = os.path.join(DATA_DIR, "fama_french.csv")
ff_to_save.to_csv(output_path_ff, index=False)

print(f"Saved equity style factor proxy data to: {output_path_ff}")
ff_to_save.head()

Saved equity style factor proxy data to: .\fama_french.csv


,Date,MKT_RF,SMB,HML,MOM,RF
0,2008-04-01,0.079966,0.0,0.0,0.0,-0.000282
1,2008-05-01,0.007332,0.0,0.0,0.0,0.001307
2,2008-06-01,-0.082364,0.0,0.0,0.0,0.002490
3,2008-07-01,-0.033246,0.0,0.0,0.0,0.001680
4,2008-08-01,-0.017283,0.0,0.0,0.0,0.000568


In [3]:
import numpy as np

DATA_DIR = "."
START_DATE = "2005-01-01"

# ---------------------------------------------------------------------
# Tickers for style premia proxies
# ---------------------------------------------------------------------

# Credit: High Yield vs Investment Grade
HY_TICKER   = "HYG"   # High Yield
IG_TICKER   = "LQD"   # Investment Grade

# Term structure: Long vs Short Treasuries
LONG_TICKER = "TLT"   # 20+ Year Treasuries
SHORT_TICKER = "SHY"  # 1–3 Year Treasuries

# Trend proxies
BOND_TICKER   = "TLT"
FX_TICKER     = "UUP"   # US Dollar Index ETF
CMDTY_TICKER  = "DBC"   # Broad commodities ETF

tickers = [HY_TICKER, IG_TICKER, LONG_TICKER, SHORT_TICKER,
           BOND_TICKER, FX_TICKER, CMDTY_TICKER]

raw = yf.download(
    tickers,
    start=START_DATE,
    interval="1mo",
    auto_adjust=True,
    progress=False
)

# We only need the Close prices
close = raw["Close"]

# Monthly returns
rets = close.pct_change().dropna()

# ---------------------------------------------------------------------
# Construct style premia proxies
# ---------------------------------------------------------------------

# Credit risk premia: HY minus IG returns
rets["CREDIT"] = rets[HY_TICKER] - rets[IG_TICKER]

# Term structure: long minus short Treasury returns
rets["TERM"] = rets[LONG_TICKER] - rets[SHORT_TICKER]

# 12-month momentum (trend) for bonds, FX and commodities
window = 12

rets["TREND_BOND"]  = rets[BOND_TICKER].rolling(window).sum()
rets["TREND_FX"]    = rets[FX_TICKER].rolling(window).sum()
rets["TREND_CMDTY"] = rets[CMDTY_TICKER].rolling(window).sum()

style = rets[["CREDIT", "TERM", "TREND_BOND", "TREND_FX", "TREND_CMDTY"]].dropna()

style_to_save = style.reset_index().rename(columns={"Date": "Date"})
output_path_style = os.path.join(DATA_DIR, "style_premia.csv")
style_to_save.to_csv(output_path_style, index=False)

print(f"Saved style premia proxy data to: {output_path_style}")
style_to_save.head()

Saved style premia proxy data to: .\style_premia.csv


Ticker,Date,CREDIT,TERM,TREND_BOND,TREND_FX,TREND_CMDTY
0,2008-04-01,0.021933,-0.016867,0.098101,-0.070240,0.436661
1,2008-05-01,0.017390,-0.022955,0.094927,-0.083375,0.508082
2,2008-06-01,-0.024328,0.024818,0.131845,-0.091457,0.604950
3,2008-07-01,0.001219,-0.008038,0.094919,-0.071986,0.488072
4,2008-08-01,-0.010241,0.023267,0.104470,-0.023969,0.441817
